In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS

In [7]:
data = pd.read_csv('./icim_5th_academy/data/parkinson_disease_speech.csv')

print(data.head(3))

  Unnamed: 0 Unnamed: 1 Baseline Features Unnamed: 3 Unnamed: 4 Unnamed: 5  \
0         id     gender               PPE        DFA       RPDE  numPulses   
1          0          1           0.85247    0.71826    0.57227        240   
2          0          1           0.76686    0.69481    0.53966        234   

         Unnamed: 6        Unnamed: 7          Unnamed: 8    Unnamed: 9  ...  \
0  numPeriodsPulses  meanPeriodPulses  stdDevPeriodPulses  locPctJitter  ...   
1               239        0.00806353            8.68E-05       0.00218  ...   
2               233       0.008258256            7.31E-05       0.00195  ...   

                Unnamed: 745               Unnamed: 746  \
0  tqwt_kurtosisValue_dec_28  tqwt_kurtosisValue_dec_29   
1                      1.562                     2.6445   
2                     1.5589                     3.6107   

                Unnamed: 747               Unnamed: 748  \
0  tqwt_kurtosisValue_dec_30  tqwt_kurtosisValue_dec_31   
1          

In [9]:
def tdaSelection(data,lens,is_min_best=True):
    data_scaled = StandardScaler().fit_transform(data)
    clusters = {}
    bins = np.histogram(lens)[1]
    bins[-1] = np.inf
    for i,f in zip(np.digitize(lens,bins),lens.index):
        if (i-1 not in clusters.keys()):
            clusters[i-1] = [f]
        else:
            clusters[i-1].append(f)
    dist = pd.DataFrame(np.arccos(abs(np.corrcoef(data_scaled.T))),index=data.columns,columns=data.columns)
    distmat = pd.DataFrame(np.zeros((len(clusters.keys()),len(clusters.keys()))),index=clusters.keys(),columns=clusters.keys())
    for i in np.arange(len(distmat)):
        for j in np.arange(i):
            distmat.iloc[i,j] = dist.loc[clusters[distmat.index[i]],clusters[distmat.index[j]]].min().min()
    distmat += distmat.T
            
    initial_cluster = max(clusters.keys()) if (is_min_best==True) else 0
    initial_feature = lens[clusters[initial_cluster]].idxmin() if (is_min_best==True) else lens[clusters[initial_cluster]].idxmax()
    
    used_clusters = [initial_cluster]
    selected_features = [initial_feature]
    
    for k in clusters.keys():
        if k==initial_cluster:
            continue
        else:
            new_cluster = distmat[used_clusters].min(axis=1).idxmax()
        used_clusters += [new_cluster]
        new_feature = lens[clusters[new_cluster]].idxmin() if (is_min_best==True) else lens[clusters[new_cluster]].idxmax()
        selected_features.append(new_feature)
        
    return selected_features,clusters,used_clusters

In [ ]:
for i in np.arrange(r):
    cluster[r] = score[i]